In [201]:
def devsml2cdpp(archivoDevsml, archivoMa, archivoEv, srcFolder):
    import xml.etree.ElementTree as ET
    from jinja2 import DictLoader, Environment, FileSystemLoader
    import os
    import re
    import numpy as np
    
    # Parametros para funciones generadoras
    PATH = './'
    TEMPLATE_ENVIRONMENT = Environment(
        autoescape=False,
        loader=FileSystemLoader(os.path.join(PATH, 'templates')),
        trim_blocks=False)
    TEMPLATE_MA         = 'template-ma.ma'
    TEMPLATE_FTOT_H     = 'template-Ftot.h'
    TEMPLATE_FTOT_CPP   = 'template-Ftot.cpp'
    TEMPLATE_FPM_H   = 'template-Fpm.h'
    TEMPLATE_FPM_CPP = 'template-Fpm.cpp'
    TEMPLATE_AUX_H   = 'template-Aux.h'
    TEMPLATE_AUX_CPP = 'template-Aux.cpp'
    
    TEMPLATE_REG_CPP = 'template-reg.cpp'
    TEMPLATE_EV      = 'template-ev.ev'

    def render_template(template_filename, context):
        return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

    tree = ET.parse(archivoDevsml)
    root = tree.getroot()

    coupled = root.find('scenario').find('coupled')
    inputPorts = coupled.find('inputs').findall('port')
    outputPorts = coupled.find('outputs').findall('port')
    atomicos = coupled.find('components').findall('atomicRef')
    links_internal = coupled.find('internal_connections').findall('connection')
    links_external = coupled.find('external_input_connections').findall('connection')

    # Atomicos
    integradores, fms, fps, fts, auxs, ctes = [], [], [], [], [], []
    for atomico in atomicos:
        if atomico.get('model') == 'QSS1':
            integradores.append(atomico)
        if atomico.get('model') == 'Ftot':
            fts.append(atomico)
        if atomico.get('model') == 'Fminus':
            fms.append(atomico)
        if atomico.get('model') == 'Fplus':
            fps.append(atomico)
        if atomico.get('model') == 'Cte':
            ctes.append(atomico)
        if atomico.get('model') == 'Aux':
            auxs.append(atomico)
            
    ######################################
    # Integradores
    integradores_ = {}
    for integrador in integradores:
        model = integrador.get('model')
        name = integrador.get('name')
        parameters = integrador.findall('parameter')
        integradores_[name] = {
            'model' : model,
            'x0' : [param.get('name') for param in parameters if param.get('name') == 'x0'][0],
            'dQRel' : [param.get('name') for param in parameters if param.get('name') == 'dQRel'][0],
            'dQMin' : [param.get('name') for param in parameters if param.get('name') == 'dQMin'][0]
        }
    ######################################
    # Ftot
    fts_ = {}
    for ftot in fts:
        model = ftot.get('model')
        name = ftot.get('name')
        ports = ftot.findall('port')
        parameters = ftot.findall('parameter')
        fts_[name] = {
            'model' : model, 'name' : name,
            'in_ports' : [port.get('name') for port in ports if port.get('type') == 'in'],
            'out_ports' : [port.get('name') for port in ports if port.get('type') == 'out'],
            'plus' : [param.get('value') for param in parameters if param.get('name') == 'plus'][0],
            'minus' : [param.get('value') for parm in parameters if param.get('name') == 'minus'][0]
        }
    ######################################
    # Fms
    fms_ = {}
    for fm in fms:
        model = fm.get('model')
        name = fm.get('name')
        ports = fm.findall('port')
        parameters = fm.findall('parameter')
        fms_[name] = {
            'model' : model, 'name' : name,
            'in_ports' : [port.get('name') for port in ports if port.get('type') == 'in'],
            'out_ports' : [port.get('name') for port in ports if port.get('type') == 'out'],
            'function' : [param.get('value') for param in parameters if param.get('name') == 'function']
        }
    ######################################
    # Fps
    fps_ = {}
    for fp in fps:
        model = fp.get('model')
        name = fp.get('name')
        ports = fp.findall('port')
        parameters = fp.findall('parameter')
        fps_[name] = {
            'model' : model, 'name' : name,
            'in_ports' : [port.get('name') for port in ports if port.get('type') == 'in'],
            'out_ports' : [port.get('name') for port in ports if port.get('type') == 'out'],
            'function' : [param.get('value') for param in parameters if param.get('name') == 'function'][0]
        }
    ######################################
    # Ctes
    ctes_ = {}
    for cte in ctes:
        model = cte.get('model')
        name = cte.get('name')
        ports = cte.findall('port')
        parameters = cte.findall('parameter')
        ctes_[name] = {
            'model' : model, 'name' : name,
            'in_ports' : [port.get('name') for port in ports if port.get('type') == 'in'],
            'out_ports' : [port.get('name') for port in ports if port.get('type') == 'out'],
            'value' : [param.get('value') for param in parameters if param.get('name') == 'value'][0]
        }
        
    ######################################
    # Auxs
    auxs_ = {}
    for aux in auxs:
        model = aux.get('model')
        name = aux.get('name')
        ports = aux.findall('port')
        parameters = aux.findall('parameter')
        #z = [param.get('function') for param in parameters if param.get('name') == 'function'][0]
        #print z
        #print len(parameters)
        auxs_[name] = {
            'model' : model, 'name' : name,
            'in_ports' : [port.get('name') for port in ports if port.get('type') == 'in'],
            'out_ports' : [port.get('name') for port in ports if port.get('type') == 'out'],
            'function' : [param.get('function') for param in parameters if param.get('name') == 'function'][0]
        }
    
    ######################################
    # Puertos de E/S
    input_ports = []
    for port in inputPorts:
        input_ports.append(port.get('name'))
    output_ports = []
    for port in outputPorts:
        output_ports.append(port.get('name'))
    ######################################
    # Conexiones internas (links_internal)
    internal_connections = []
    for conn in links_internal:
        comp_from = conn.get('component_from')
        comp_to   = conn.get('component_to')
        port_from = conn.get('port_from')
        port_to   = conn.get('port_to')
        internal_connections.append({
            'component_from': comp_from,
            'component_to'  : comp_to,
            'port_from'     : port_from,
            'port_to'       : port_to
        })
    # Conexiones externas (links_external)
    external_connections = []
    for conn in links_external:
        comp_from = conn.get('component_from')
        comp_to   = conn.get('component_to')
        port_from = conn.get('port_from')
        port_to   = conn.get('port_to')
        external_connections.append({
            'component_from': comp_from,
            'component_to'  : comp_to,
            'port_from'     : port_from,
            'port_to'       : port_to
        })
    #####################################
    context = {
        'internal_connections' : internal_connections,
        'external_connections' : external_connections,
        'input_ports' : input_ports,
        'output_ports' : output_ports,
        
        'integradores_' : integradores_,
        'fts_' : fts_,
        'fms_' : fms_,
        'fps_' : fps_,
        'ctes_' : ctes_,
        'auxs_' : auxs_
    }
    
    
    
    #print(context)
    #####################################
    # Guardar el .ma en output
    with open(archivoMa, 'w') as f:
        f.write(render_template(TEMPLATE_MA,context))
        
    ############### GENERATE .H's AND .CPP's ###############
    # Ftot's
    #print(fts_)
    for ftName, ftAttr in fts_.iteritems():
        ft_inPorts = []
        ft_outPorts = []
        for port_name in ftAttr['in_ports']:
            ft_inPorts.append(port_name)
        for port_name in ftAttr['out_ports']:
            ft_outPorts.append(port_name)
        ft_context = {
            'name' : ftName, 'nameUpper' : ftName.upper(),
            'ft_inPorts' : ft_inPorts, 'ft_outPorts' : ft_outPorts, 
            'values' : [inPort for inPort in ft_inPorts],
            'model' : ftAttr['model'], 'modelUpper' : ftAttr['model'].upper() }
        ##################
        # GUARDAR FTOT'S
        with open(srcFolder + ftName + '.h', 'w') as f:
            f.write(render_template(TEMPLATE_FTOT_H, ft_context))
        with open(srcFolder + ftName + '.cpp', 'w') as f:
            f.write(render_template(TEMPLATE_FTOT_CPP, ft_context))
            
    
    #print(auxs_)
    # Aux's
    for auxName, auxAttr in auxs_.iteritems():
        aux_inPorts = []
        aux_outPorts = []
        for port_name in auxAttr['in_ports']:
            aux_inPorts.append(port_name)
        for port_name in auxAttr['out_ports']:
            aux_outPorts.append(port_name)
        aux_context = {
            'name' : auxName, 'nameUpper' : auxName.upper(),
            'model' : auxAttr['model'], 'modelUpper' : auxAttr['model'].upper(),
            'aux_inPorts' : aux_inPorts, 'aux_outPorts' : aux_outPorts,
            'values' : [inPort[3:] for inPort in aux_inPorts],
            'function' : auxAttr['function']
        }
        ##################
        # GUARDAR AUX'S
        with open(srcFolder + auxName + '.h', 'w') as f:
            f.write(render_template(TEMPLATE_AUX_H, aux_context))
        with open(srcFolder + auxName + '.cpp', 'w') as f:
            f.write(render_template(TEMPLATE_AUX_CPP, aux_context))
    
    # Fm's + Fp's
    for fmName, fmAttr in fms_.iteritems():
        fm_inPorts = []
        fm_outPorts = []
        for port_name in fmAttr['in_ports']:
            fm_inPorts.append(port_name)
        for port_name in fmAttr['out_ports']:
            fm_outPorts.append(port_name)
        fm_context = {
            'name' : fmName, 'nameUpper' : fmName.upper(),
            'model' : fmAttr['model'], 'modelUpper' : fmAttr['model'].upper(),
            'fpm_inPorts' : fm_inPorts, 'fpm_outPorts' : fm_outPorts, 
            'values' : [inPort[3:] for inPort in fm_inPorts],
            'function' : fmAttr['function']
        }
        ##################
        # GUARDAR FMINUS'S
        with open(srcFolder + fmName + '.h', 'w') as f:
            f.write(render_template(TEMPLATE_FPM_H, fm_context))
        with open(srcFolder + fmName + '.cpp', 'w') as f:
            f.write(render_template(TEMPLATE_FPM_CPP, fm_context))

    for fpName, fpAttr in fps_.iteritems():
        fp_inPorts = []
        fp_outPorts = []
        for port_name in fpAttr['in_ports']:
            fp_inPorts.append(port_name)
        for port_name in fpAttr['out_ports']:
            fp_outPorts.append(port_name)
        fp_context = {
            'name' : fpName, 'nameUpper' : fpName.upper(),
            'model' : fpAttr['model'], 'modelUpper' : fpAttr['model'].upper(),
            'fpm_inPorts' : fp_inPorts, 'fpm_outPorts' : fp_outPorts,
            'values' : [inPort[3:] for inPort in fp_inPorts],
            'function' : fpAttr['function']
        }
        ##################
        # GUARDAR FPLUS'S
        with open(srcFolder + fpName + '.h', 'w') as f:
            f.write(render_template(TEMPLATE_FPM_H, fp_context))
        with open(srcFolder + fpName + '.cpp', 'w') as f:
            f.write(render_template(TEMPLATE_FPM_CPP, fp_context))
    #print fps_

    #######
    # Generate 'Register File'
    reg_context = { 'models' : {} }
    for ftName, ftAttr in fts_.iteritems():
        reg_context['models'].update({ftName : { 'modelUpper' : ftAttr['name'].upper() }})
    for fpName, fpAttr in fps_.iteritems(): 
        reg_context['models'].update({fpName :  { 'modelUpper' : fpAttr['name'].upper() }})
    for fmName, fmAttr in fms_.iteritems():
        reg_context['models'].update({fmName : { 'modelUpper' : fmAttr['name'].upper() }})
    for auxName, auxAttr in auxs_.iteritems():
        reg_context['models'].update({auxName : { 'modelUpper' : auxAttr['name'].upper() }})
    with open(srcFolder + 'reg.cpp', 'w') as f:
        f.write(render_template(TEMPLATE_REG_CPP, reg_context))

    #######
    # Generate 'Events (.ev) File'
    ev_context = { 'inputs' : {} }
    for cteName, cteAttr in ctes_.iteritems():
        ev_context['inputs']['in_' + cteName] = cteAttr['value']
    #print ev_context
    with open(archivoEv, 'w') as f:
        f.write(render_template(TEMPLATE_EV,ev_context))

In [202]:
#archivoDevsml = 'goodwin/goodwin-devsml-traduccion.xml'
#archivoMa = 'goodwin/goodwin-traduccion.ma'
#archivoEv = 'goodwin/goodwin.ev'
#devsml2cdpp(archivoDevsml, archivoMa, archivoEv, 'goodwin/src/')

archivoDevsml = 'sir/sir-devsml-traduccion.xml'
archivoMa = 'sir/sir-traduccion.ma'
archivoEv = 'sir/sir.ev'
devsml2cdpp(archivoDevsml, archivoMa, archivoEv, 'sir/src/')